AUTHOR: SPARSHSAH

# BAB: The Beauty of Levered Returns

I'm going to keep things simple here for illustration. But MVO (mean-variance optimation) and BAB (betting against beta) are extremely well-developed concepts in quantitative finance (both stock selection and global macro), and you really ought to study them for a fuller picture.

## Givens

* You have $\$100$ to invest.
* You can choose between two pools of 1-year bonds, Pool A and Pool B. (You may not split your allocation between the two -- That is diversification, which is another powerful concept but not the point here.)
* Pool A has coupon 30% and ex-ante annualized loss rate 15% (with stress loss 30%). This gives you a "spread" of 30% - 15% = 15%.
* Pool B has coupon 20% and ex-ante annualized loss rate 8% (with stress loss 16%). This gives you a "spread" of 20% - 8% = 12%.
* Cost of leverage is 5% per year.

## Strategy

What should you choose? Maybe Pool A -- it gives you a better spread. But maybe Pool B -- it's much safer.

Let's suppose that you're comfortable with some risk, though, so you choose Pool A to get that extra 3pts of spread. So, you put your $\$100$ into Pool A. (Ignore the not-quite-right arithmetic in the next part, the right way to do it is in terms of Sharpe and ER and volatility, but those are a bit jargon that are simple but sound unnecessarily fancy and intimidating. It's ok though, ER is roughly proportional to spread, and volatility---if you imagine each bond's default indicator as a Bernoulli flip---is roughly proportional to default probability which is roughly proportional to annualized loss rate.) You expect to earn $\$30$ in coupon while losing $\$15$ to losses, netting $\$15$ in profit. So, you expect to end the year with $\$115$.

But now let's allow leverage. What if you take your $\$100$ and invest it in Pool B. But then, you go to the bank, and borrow another $\$100$, and invest _that_ into Pool B too. So now you have $\$200$ in Pool B. You expect to earn $\$20 \cdot 2 = \$40$ in coupon, while losing $\$8 \cdot 2 = \$16$ to losses, netting $\$24$ in profit. So, you expect to end the year with $\$224$. But of course now you have to return the $\$100$ you borrowed to the bank, _plus_ $\$5$ of interest, which means you _really_ expect to end the year with $\$119$. So, rather than $\$15$ of losses you expect $\$16$ of losses, but in exchange you expect to end the year $\$4$ richer than with Pool A! Not bad.

# Asymmetry: The Vexatious Nature of Levered Returns

First of all, note that maintaining leverage over some given horizon requires you to dynamically rebalance to target.

For example, suppose SPXL, Direxion's Daily 3x-Levered Bull SPX ETF, has USD1 NAV (USD1 client assets + USD2 borrowed money - USD2 owed on the loan), and hence invests USD3 in the SP500. Then suppose today the SP500 return 1pct (assume zero interest rates), growing SPXL's NAV to USD1.03.

This means Direxion must go back to the bank and borrow an additional USD0.06, because now (assuming they didn't buy or sell any of the shares, meaning that conveniently their holdings naturally evolved to reflect the new composition of the SP500, which is a convenient feature of market-cap-weighted benchmarks and indeed a cornerstone of truly "passive" investment strategies) they are technically at 2.94x=USD3.03/USD1.03 leverage which is slightly less than their 3x stated target.

Second of all, take a look at the below. Levering up to 1.20x seems like a good tradeoff: If you consistently win, you overall win fully 1.473x as much; whereas if you consistently lose, you overall lose only 1.068x as much. But levering down to 0.80x seems like a bad tradeoff: If you consistently win, you overall win only 0.657x as much; whereas if you consistently lose, you overall lose fully 0.905x as much. Note that the "contrapositive" therefore also holds: Based on this logic, levering up from 0.80x to 1.00x seems like a good tradeoff. The upshot is that deleveraging as a risk-reduction tactic might dampen your wins more than expected, while simultaneously dampening your losses less than expected.

For a more optimistic example of a similar financial "convexity" phenomenon, cf Aaron Brown at https://www.bloomberg.com/opinion/articles/2020-05-26/twitter-spat-over-market-risks-is-a-teaching-moment.

In [1]:
ANN = 261  # financial calendar daycount (# of trading days per year)
def cmp_pnl(lev=1, ltv=0.10, ann=ANN, sign=+1, std=1, dur=ANN):
    """
    Compare `lev`x-levered vs 1x-levered pnl
    if we realize `sign*std`STD pnl
    (at `ltv` annualized long-term vol)
    for `dur`days in a row.
    """
    def get_pnl(lev):
        print ("\tleverage = {lev}x".format(lev=lev))
        daily_pnl = lev * sign * std * ltv / ann**0.5
        px = (1 + daily_pnl)**dur
        pnl = px - 1
        print("\t\tfinal price index = ${px}".format(px=round(px, 2)))
        print("\t\tfinal pnl = {pnl}%".format(pnl=round(100*pnl, 1)))
        return pnl
    unlev_pnl = get_pnl(lev=1)
    lev_pnl = get_pnl(lev=lev)
    print("\t>>>> ratio = {} <<<<".format(round(lev_pnl / unlev_pnl, 3)))

for lev in 1.20, 0.80:
    for sign in +1, -1:
        print("SIGN = {sign}".format(sign=sign))
        cmp_pnl(lev=lev, sign=sign)
    print()

SIGN = 1
	leverage = 1x
		final price index = $5.01
		final pnl = 400.6%
	leverage = 1.2x
		final price index = $6.9
		final pnl = 590.0%
	>>>> ratio = 1.473 <<<<
SIGN = -1
	leverage = 1x
		final price index = $0.2
		final pnl = -80.2%
	leverage = 1.2x
		final price index = $0.14
		final pnl = -85.7%
	>>>> ratio = 1.068 <<<<

SIGN = 1
	leverage = 1x
		final price index = $5.01
		final pnl = 400.6%
	leverage = 0.8x
		final price index = $3.63
		final pnl = 263.0%
	>>>> ratio = 0.657 <<<<
SIGN = -1
	leverage = 1x
		final price index = $0.2
		final pnl = -80.2%
	leverage = 0.8x
		final price index = $0.27
		final pnl = -72.6%
	>>>> ratio = 0.905 <<<<

